In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset,CompasDataset

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# TODO: change the import method
import sys
import os
repo_root = os.path.dirname(os.getcwd())
sys.path.insert(0, repo_root)
repair_folder = os.path.join(repo_root, "humancompatible", "repair")
sys.path.insert(0, repair_folder)
from humancompatible.repair.cost import *
from humancompatible.repair.coupling_utils import *
from humancompatible.repair.data_analysis import *
from humancompatible.repair.group_blind_repair import *
from humancompatible.repair.metrics import *

In [4]:
import os
path=os.path.dirname(os.getcwd())

In [5]:
class ROCpostprocess:
    def __init__(self,X_val,y_val,var_list,prediction_model,favorable_label):
        self.X_val =X_val
        self.y_val =y_val
        self.model = prediction_model
        self.positive_index = 1 # positive label
        self.var_list = var_list
        self.var_dim=len(self.var_list)
        self.ROC = self.buildROCusingval()
        self.favorable_label = favorable_label

    def buildbinarydata(self,X,y):
        df=pd.DataFrame(np.concatenate((X,y.reshape(-1,1)), axis=1),columns=self.var_list+['S','W','Y'])
        binaryLabelDataset = BinaryLabelDataset(
                            # favorable_label=self.favorable_label,
                            # unfavorable_label=0,
                            df=df[self.var_list+['S','W','Y']], #df_test.drop('X',axis=1), #[x_list+['S','W','Y']],
                            label_names=['Y'],
                            instance_weights_name=['W'],
                            protected_attribute_names=['S'],
                            privileged_protected_attributes=[np.array([1.0])],
                            unprivileged_protected_attributes=[np.array([0.])])
        return binaryLabelDataset,df

    def buildROCusingval(self):
        dataset_val = self.buildbinarydata(self.X_val,self.y_val)[0]
        dataset_val_pred = dataset_val.copy(deepcopy=True)
        dataset_val_pred.scores = self.model.predict_proba(dataset_val.features[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        privileged_groups = [{'S': 1}]
        unprivileged_groups = [{'S': 0}]
        # Metric used (should be one of allowed_metrics)
        metric_name = "Statistical parity difference"
        # Upper and lower bound on the fairness metric used
        metric_ub = 0.05
        metric_lb = -0.05
        ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                        privileged_groups=privileged_groups, 
                                        low_class_thresh=0.01, high_class_thresh=0.99,
                                        num_class_thresh=50, num_ROC_margin=10,
                                        metric_name=metric_name,
                                        metric_ub=metric_ub, metric_lb=metric_lb)
        ROC = ROC.fit(dataset_val, dataset_val_pred)
        print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
        print("Optimal ROC margin = %.4f" % ROC.ROC_margin)
        return ROC

    def postprocess(self,X_test,y_test,tv_origin): # the tv distance won't change
        dataset_test_pred,df_test = self.buildbinarydata(X_test,y_test) #.copy(deepcopy=True)
        dataset_test_pred.scores = self.model.predict_proba(X_test[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        dataset_test_pred_transf = self.ROC.predict(dataset_test_pred)
        y_pred = dataset_test_pred_transf.labels
        # return dataset_test_pred_transf.convert_to_dataframe()[0]

        di = DisparateImpact_postprocess(df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(df_test['Y'], y_pred, average='macro',sample_weight=df_test['W'])
        f1_micro = f1_score(df_test['Y'], y_pred, average='micro',sample_weight=df_test['W'])
        f1_weighted = f1_score(df_test['Y'], y_pred, average='weighted',sample_weight=df_test['W'])
        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv_origin,'method':'ROC'})
        return new_row.to_frame().T

In [6]:
from humancompatible.repair.proj_postprocess import Projpostprocess

In [7]:
pa = 'race'
label_map = {1.0: 'Did recid.', 0.0: 'No recid.'}
protected_attribute_maps = {1.0: 'Caucasian', 0.0: 'African-American'}
favorable_label = 0
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
cd = CompasDataset(protected_attribute_names=[pa],privileged_classes=[['Caucasian'],[1]], 
                    metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
                    features_to_drop=['age', 'sex', 'c_charge_desc'])
train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

K=200
e=0.01
thresh=0.05

messydata=cd.convert_to_dataframe()[0]
messydata=messydata.rename(columns={pa:'S',cd.label_names[0]:'Y'})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('category')
messydata['W']=cd.instance_weights
X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.1:
        x_list+=[key]        
tv_dist

{'juv_fel_count': np.float64(0.03210337325453563),
 'juv_misd_count': np.float64(0.04323143324022939),
 'juv_other_count': np.float64(0.021763780679615215),
 'priors_count': np.float64(0.12622233191661625),
 'age_cat=25 - 45': np.float64(0.054431947619680315),
 'age_cat=Greater than 45': np.float64(0.13519019921101838),
 'age_cat=Less than 25': np.float64(0.08075825159133806),
 'c_charge_degree=F': np.float64(0.07840757396162046),
 'c_charge_degree=M': np.float64(0.07840757396162046)}

In [8]:
x_list

['priors_count', 'age_cat=Greater than 45']

In [9]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.01,0.1),theta=1e-2)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv(path+'/data/report_postprocess_compas_'+str(pa)+'.csv',index=None)

Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.2300
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7300
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7500
Optimal ROC margin = 0.0000


c:\personal\work\repair\humancompatible\repair\group_blind_repair.py:284: RuntimeWarning: overflow encountered in exp
  tmp.item(i, j) * V.item(i) * np.exp(-z * V.item(i)) for i in I
c:\personal\work\repair\humancompatible\repair\group_blind_repair.py:287: RuntimeWarning: overflow encountered in exp
  tmp.item(i, j) * (V.item(i) ** 2) * np.exp(-z * V.item(i)) for i in I
c:\personal\work\repair\humancompatible\repair\group_blind_repair.py:318: RuntimeWarning: invalid value encountered in scalar divide
  b = a - fun(a) / dfun(a)
c:\personal\work\repair\humancompatible\repair\data_analysis.py:28: RuntimeWarning: invalid value encountered in divide
  rdist['x'] = np.array([pivot[i] for i in x_range]) / total
c:\personal\work\repair\humancompatible\repair\data_analysis.py:37: RuntimeWarning: invalid value encountered in divide
  rdist['x_0'] = np.array(
c:\personal\work\repair\humancompatible\repair\data_analysis.py:48: RuntimeWarning: invalid value encountered in divide
  rdist['x_1'] = np

Optimal classification threshold (with fairness constraints) = 0.7300
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.2300
Optimal ROC margin = 0.0000


In [10]:
# report.to_csv(path+'/data/report_postprocess_compas_'+str(pa)+'_'+str(thresh)+'.csv',index=None)

In [11]:
report

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.772376,0.667954,0.674362,0.672232,0.187271,origin
1,0.683099,0.666246,0.666262,0.666459,0.186524,unconstrained
2,1.229719,0.623534,0.624139,0.624934,0.000001,barycentre
3,0.988478,0.555826,0.566626,0.549402,0.020226,partial_0.001
4,0.969396,0.44476,0.578777,0.470061,0.187271,ROC
5,0.750578,0.676627,0.680437,0.679343,0.196867,origin
6,0.731857,0.667845,0.668287,0.668782,0.196117,unconstrained
7,1.220334,0.630212,0.630215,0.630143,0.0,barycentre
8,0.884235,0.572482,0.584042,0.567044,0.020912,partial_0.001
9,0.316086,0.445468,0.519644,0.429779,0.196867,ROC


In [12]:
valpost = Projpostprocess(X_val,y_val,x_list,var_list,clf,K,e,'auto',linspace_range=(0.01,0.1),theta=1e-2)
valpost.thresh

Optional threshold =  [0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 ]
Disparate Impact =  [1.07353072 1.09561814 1.09561814 1.09715353 1.09215511 1.19962117
 1.30701583 1.30701583 1.30701583 1.30701583]
f1 scores =  [0.5622166  0.58781641 0.58781641 0.58306094 0.58133219 0.63968347
 0.64711929 0.64711929 0.64711929 0.64711929]


np.float64(0.07)

In [13]:
# Compute average feature importance
importance=[]
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    importance.append(list(clf.feature_importances_))
importance=np.array(importance)
print("features", var_list)
print("mean importances", importance.mean(axis=0))

features ['juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'age_cat=25 - 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M']
mean importances [0.02715225 0.06211627 0.0899227  0.55570939 0.03352981 0.07705334
 0.09128189 0.03173411 0.03150024]
